1) Download all symbols and their weights in the S&P 500.

In [2]:
import pandas as pd
import requests
import cloudscraper
from io import StringIO
from bs4 import BeautifulSoup

url = "https://www.slickcharts.com/sp500"
scraper = cloudscraper.create_scraper()
soup = BeautifulSoup(scraper.get(url).text, 'html.parser')
sap_table = soup.find('table', {'class':"table"})
sap_list = pd.read_html(StringIO(str(sap_table)))[0]
sap = pd.DataFrame(sap_list)

sap = sap[["Company", "Symbol", "Portfolio%"]]
sap

,Company,Symbol,Portfolio%
0,Microsoft Corp,MSFT,7.17%
1,Apple Inc.,AAPL,6.16%
2,Nvidia Corp,NVDA,4.56%
3,Amazon.com Inc,AMZN,3.75%
4,"Meta Platforms, Inc. Class A",META,2.54%
...,...,...,...
498,Zions Bancorporation N.a.,ZION,0.01%
499,V.F. Corporation,VFC,0.01%
500,Paramount Global Class B,PARA,0.01%
501,Fox Corporation Class B,FOX,0.01%


2) Remove any equities you don't want to hold

In [6]:
banned_tickers = ("TSLA", "FOX", "NWS")
sap = sap[~sap["Symbol"].isin(banned_tickers)].copy()
sap

,Company,Symbol,Portfolio%,PortfolioFraction
0,Microsoft Corp,MSFT,7.17%,0.0500
1,Apple Inc.,AAPL,6.16%,0.0500
2,Nvidia Corp,NVDA,4.56%,0.0456
3,Amazon.com Inc,AMZN,3.75%,0.0375
4,"Meta Platforms, Inc. Class A",META,2.54%,0.0254
...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.0001
497,Whirlpool Corp.,WHR,0.01%,0.0001
498,Zions Bancorporation N.a.,ZION,0.01%,0.0001
499,V.F. Corporation,VFC,0.01%,0.0001


3. Also optionally clip weights

In [8]:
sap["PortfolioFraction"] = sap["Portfolio%"].str.strip("%").astype('float') / 100
sap["PortfolioFraction"] = sap["PortfolioFraction"].clip(lower=0, upper=0.05)
sap

,Company,Symbol,Portfolio%,PortfolioFraction
0,Microsoft Corp,MSFT,7.17%,0.0500
1,Apple Inc.,AAPL,6.16%,0.0500
2,Nvidia Corp,NVDA,4.56%,0.0456
3,Amazon.com Inc,AMZN,3.75%,0.0375
4,"Meta Platforms, Inc. Class A",META,2.54%,0.0254
...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.0001
497,Whirlpool Corp.,WHR,0.01%,0.0001
498,Zions Bancorporation N.a.,ZION,0.01%,0.0001
499,V.F. Corporation,VFC,0.01%,0.0001


4. Reweight now that some of the tickers were removed or clipped

In [11]:
increase_prec = 1 - sap["PortfolioFraction"].sum() + 1
sap["PortfolioFraction"] = sap["PortfolioFraction"] * increase_prec
sap

,Company,Symbol,Portfolio%,PortfolioFraction
0,Microsoft Corp,MSFT,7.17%,0.052626
1,Apple Inc.,AAPL,6.16%,0.052626
2,Nvidia Corp,NVDA,4.56%,0.047995
3,Amazon.com Inc,AMZN,3.75%,0.039470
4,"Meta Platforms, Inc. Class A",META,2.54%,0.026734
...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.000105
497,Whirlpool Corp.,WHR,0.01%,0.000105
498,Zions Bancorporation N.a.,ZION,0.01%,0.000105
499,V.F. Corporation,VFC,0.01%,0.000105


5) Import a dataframe of your current equity holdings & cash. You're a bit on your own here but my brockage allows me to download a CSV of all of my holding. I'm going to assume you have a simplified version of that.

In [12]:
#Write out some test data
with open('holdings.csv', 'w') as f:
    f.write("""symbol,shares
AMZN,10
NVDA,1
MSFT,1
YUM,100
ZTS,10
GME,100""")

In [13]:
holdings = pd.read_csv('holdings.csv')
holdings

,symbol,shares
0,AMZN,10
1,NVDA,1
2,MSFT,1
3,YUM,100
4,ZTS,10
5,GME,100


In [14]:
cash_to_invest = 10000

6) Get last share price of all of holdings and compute our total assests to be invested

In [15]:
import yfinance as yf

def get_current_price(symbol):
    ticker = yf.Ticker(symbol.replace(".","-"))
    todays_data = ticker.history(period='1d')
    return todays_data['Close'].iloc[0]

holdings["last$"] = holdings["symbol"].apply(get_current_price)
holdings["total$"] = holdings["shares"] * holdings["last$"]

total_assets = holdings['total$'].sum() + cash_to_invest
total_assets
print(f"total_assets:", total_assets)
holdings

total_assets: 30298.690364837646


,symbol,shares,last$,total$
0,AMZN,10,177.580002,1775.800018
1,NVDA,1,852.369995,852.369995
2,MSFT,1,414.920013,414.920013
3,YUM,100,138.550003,13855.000305
4,ZTS,10,187.860001,1878.600006
5,GME,100,15.220000,1522.000027


7) Also get lastest share price from all S&P 500 symbols

In [16]:
sap["last$"] = sap["Symbol"].apply(get_current_price)

In [17]:
sap

,Company,Symbol,Portfolio%,PortfolioFraction,last$
0,Microsoft Corp,MSFT,7.17%,0.052626,414.920013
1,Apple Inc.,AAPL,6.16%,0.052626,175.100006
2,Nvidia Corp,NVDA,4.56%,0.047995,852.369995
3,Amazon.com Inc,AMZN,3.75%,0.039470,177.580002
4,"Meta Platforms, Inc. Class A",META,2.54%,0.026734,498.190002
...,...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.000105,120.830002
497,Whirlpool Corp.,WHR,0.01%,0.000105,106.550003
498,Zions Bancorporation N.a.,ZION,0.01%,0.000105,39.169998
499,V.F. Corporation,VFC,0.01%,0.000105,15.730000


8) Compute shares we should own to match give our current investable assets

In [18]:
sap["$NeededToMatch"] = sap["PortfolioFraction"] * total_assets
sap["SharesToMatch"] = (sap["$NeededToMatch"] / sap["last$"]).astype('int') # assume no factional shares
sap

,Company,Symbol,Portfolio%,PortfolioFraction,last$,$NeededToMatch,SharesToMatch
0,Microsoft Corp,MSFT,7.17%,0.052626,414.920013,1594.500072,3
1,Apple Inc.,AAPL,6.16%,0.052626,175.100006,1594.500072,9
2,Nvidia Corp,NVDA,4.56%,0.047995,852.369995,1454.184065,1
3,Amazon.com Inc,AMZN,3.75%,0.039470,177.580002,1195.875054,6
4,"Meta Platforms, Inc. Class A",META,2.54%,0.026734,498.190002,810.006036,1
...,...,...,...,...,...,...,...
496,"Mohawk Industries, Inc.",MHK,0.01%,0.000105,120.830002,3.189000,0
497,Whirlpool Corp.,WHR,0.01%,0.000105,106.550003,3.189000,0
498,Zions Bancorporation N.a.,ZION,0.01%,0.000105,39.169998,3.189000,0
499,V.F. Corporation,VFC,0.01%,0.000105,15.730000,3.189000,0


9) Join to our current holdings

In [19]:
pd.set_option('display.max_rows', None)
movesWithHoldings = sap.merge(holdings[["symbol", "shares"]], left_on='Symbol', right_on='symbol', how='outer')
movesWithHoldings["shares"] = movesWithHoldings["shares"].fillna(0)
movesWithHoldings["SharesToMatch"] = movesWithHoldings["SharesToMatch"].fillna(0)
movesWithHoldings.loc[movesWithHoldings["Symbol"].isna(),"Symbol"] = movesWithHoldings[movesWithHoldings["Symbol"].isna()]["symbol"]
movesWithHoldings

,Company,Symbol,Portfolio%,PortfolioFraction,last$,$NeededToMatch,SharesToMatch,symbol,shares
0,Agilent Technologies Inc.,A,0.09%,0.000947,142.860001,28.701001,0.0,NaN,0.0
1,American Airlines Group Inc.,AAL,0.02%,0.000211,14.810000,6.378000,0.0,NaN,0.0
2,Apple Inc.,AAPL,6.16%,0.052626,175.100006,1594.500072,9.0,NaN,0.0
3,Abbvie Inc.,ABBV,0.72%,0.007578,177.050003,229.608010,1.0,NaN,0.0
4,"Airbnb, Inc. Class A",ABNB,0.16%,0.001684,158.089996,51.024002,0.0,NaN,0.0
5,Abbott Laboratories,ABT,0.48%,0.005052,120.040001,153.072007,1.0,NaN,0.0
6,Arch Capital Group Ltd,ACGL,0.08%,0.000842,87.139999,25.512001,0.0,NaN,0.0
7,Accenture Plc,ACN,0.55%,0.005789,383.709991,175.395008,0.0,NaN,0.0
8,Adobe Inc.,ADBE,0.59%,0.006210,567.940002,188.151008,0.0,NaN,0.0
9,"Analog Devices, Inc.",ADI,0.22%,0.002316,194.660004,70.158003,0.0,NaN,0.0


11) Find the shares we need to match the S&P 500 prop

In [20]:
movesWithHoldings["SharesDifference"] = movesWithHoldings["SharesToMatch"] - movesWithHoldings["shares"]
movesWithHoldings

,Company,Symbol,Portfolio%,PortfolioFraction,last$,$NeededToMatch,SharesToMatch,symbol,shares,SharesDifference
0,Agilent Technologies Inc.,A,0.09%,0.000947,142.860001,28.701001,0.0,NaN,0.0,0.0
1,American Airlines Group Inc.,AAL,0.02%,0.000211,14.810000,6.378000,0.0,NaN,0.0,0.0
2,Apple Inc.,AAPL,6.16%,0.052626,175.100006,1594.500072,9.0,NaN,0.0,9.0
3,Abbvie Inc.,ABBV,0.72%,0.007578,177.050003,229.608010,1.0,NaN,0.0,1.0
4,"Airbnb, Inc. Class A",ABNB,0.16%,0.001684,158.089996,51.024002,0.0,NaN,0.0,0.0
5,Abbott Laboratories,ABT,0.48%,0.005052,120.040001,153.072007,1.0,NaN,0.0,1.0
6,Arch Capital Group Ltd,ACGL,0.08%,0.000842,87.139999,25.512001,0.0,NaN,0.0,0.0
7,Accenture Plc,ACN,0.55%,0.005789,383.709991,175.395008,0.0,NaN,0.0,0.0
8,Adobe Inc.,ADBE,0.59%,0.006210,567.940002,188.151008,0.0,NaN,0.0,0.0
9,"Analog Devices, Inc.",ADI,0.22%,0.002316,194.660004,70.158003,0.0,NaN,0.0,0.0


12) Find tickers that we need to sell

In [21]:
movesWithHoldings[movesWithHoldings["SharesDifference"] < 0][["Symbol", "SharesDifference"]] 

,Symbol,SharesDifference
32,AMZN,-4.0
205,GME,-100.0
496,YUM,-100.0
500,ZTS,-10.0


13) Find tickers that we need to buy

In [22]:
movesWithHoldings[movesWithHoldings["SharesDifference"] > 0][["Symbol", "SharesDifference"]]

,Symbol,SharesDifference
2,AAPL,9.0
3,ABBV,1.0
5,ABT,1.0
26,AMCR,1.0
27,AMD,1.0
51,BAC,4.0
67,BMY,1.0
69,BRK.B,1.0
71,BSX,1.0
75,C,1.0
